In [2]:
from box import ConfigBox

In [4]:
exaple = ConfigBox({
    "key": "value",
    "key2": 123,
    "key3": [1, 2, 3],
    "key4": (1, 2, 3),})

In [5]:
exaple.key2

123

In [11]:
from ensure import ensure_annotations
@ensure_annotations
def get_prod(x:int ,y:int)->int:
    return x * y
    

In [12]:
get_prod(2, 3)

6

In [14]:
get_prod(2, "3")

EnsureError: Argument y of type <class 'str'> to <function get_prod at 0x0000021544C70040> does not match annotation type <class 'int'>